In [5]:
!pip install opencv-python flask Pillow

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -- ------------------------------------- 2.6/39.5 MB 12.5 MB/s eta 0:00:03
   ----- ---------------------------------- 5.2/39.5 MB 12.3 MB/s eta 0:00:03
   ------ --------------------------------- 6.3/39.5 MB 9.9 MB/s eta 0:00:04
   --------- ------------------------------ 8.9/39.5 MB 10.4 MB/s eta 0:00:03
   ----------- ---------------------------- 11.5/39.5 MB 10.9 MB/s eta 0:00:03
   -------------- ------------------------- 14.2/39.5 MB 11.1 MB/s eta 0:00:03
   ---------------- ----------------------- 16.8/39.5 MB 11.2 MB/s eta 0:00:03
   ------------------ --------------------- 18.4/39.5 MB 10.7 MB/s eta 0:00:02
   --------------------- ------------------ 21.0/39.5 MB 10.9 MB/s eta 0:00:02
   ----------------------- ---------------- 23.6/39.5 MB 11.1 MB/s eta 0:00:02
   -------------------------- ------------- 26.2/39.5 MB 11.1 MB/s eta 0:00:02
   ----------------------------- ---------- 28.8/39.5 MB 11.3 MB/s


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
FocusNet-LC 웹 인터페이스 - 폐암 진단, Grad-CAM 시각화 및 개인화된 건강 권고
"""

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from flask import Flask, request, render_template, jsonify, send_file
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import io
import base64
from PIL import Image
from scipy import ndimage
from skimage import morphology, measure, segmentation
from lungmask import LMInferer
import SimpleITK as sitk

h5_path = "lung_model.h5"

# 설정
IMG_SIZE = 512
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# 업로드 폴더 생성
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# 클래스 라벨
CLASS_NAMES = ['Normal', 'Benign', 'Malignant']
CLASS_COLORS = ['#28a745', '#ffc107', '#dc3545']  # 초록, 노랑, 빨강

def generate_health_recommendations(prediction_result, survey_data):
    """
    진단 결과와 설문조사 결과에 따른 개인화된 건강 권고사항 생성
    
    Args:
        prediction_result: 예측 결과 딕셔너리
        survey_data: 설문조사 데이터 딕셔너리
    
    Returns:
        recommendations: 건강 권고사항 딕셔너리
    """
    
    primary_prediction = prediction_result['prediction']
    confidence = prediction_result['confidence']
    all_predictions = prediction_result['all_predictions']
    
    is_smoker = survey_data.get('smoking', False)
    exercise_frequency = survey_data.get('exercise', 0)  # 0: 안함, 1: 가끔, 2: 정기적
    age = survey_data.get('age', 40)
    
    recommendations = {
        'priority': 'low',  # low, medium, high, urgent
        'smoking_advice': [],
        'exercise_advice': [],
        'medical_advice': [],
        'lifestyle_advice': [],
        'followup_advice': []
    }
    
    # 기본 위험도 계산
    benign_prob = all_predictions.get('Benign', 0)
    malignant_prob = all_predictions.get('Malignant', 0)
    risk_score = benign_prob + (malignant_prob * 2)
    
    if primary_prediction == 'Normal':
        if risk_score > 20:  # Normal이지만 다른 확률이 높은 경우
            recommendations['priority'] = 'medium'
            recommendations['medical_advice'].append(
                f"현재는 정상으로 판단되지만, 양성({benign_prob:.1f}%) 또는 악성({malignant_prob:.1f}%) 가능성도 보입니다."
            )
            recommendations['followup_advice'].append("6개월 후 재검사를 권장합니다.")
        else:
            recommendations['priority'] = 'low'
            recommendations['medical_advice'].append("현재 폐 상태는 정상으로 보입니다.")
            recommendations['followup_advice'].append("1년 후 정기 검진을 받으시기 바랍니다.")
            
    elif primary_prediction == 'Benign':
        recommendations['priority'] = 'medium'
        recommendations['medical_advice'].append("양성 병변이 발견되었습니다. 추가 정밀검사가 필요합니다.")
        recommendations['medical_advice'].append("호흡기내과 전문의 상담을 받으시기 바랍니다.")
        recommendations['followup_advice'].append("3개월 후 추적 검사가 필요합니다.")
        
    elif primary_prediction == 'Malignant':
        recommendations['priority'] = 'urgent'
        recommendations['medical_advice'].append("⚠️ 악성 병변 의심 소견이 있습니다.")
        recommendations['medical_advice'].append("🏥 즉시 종합병원 호흡기내과 또는 흉부외과 진료를 받으시기 바랍니다.")
        recommendations['medical_advice'].append("추가 정밀검사(CT, PET-CT, 조직검사 등)가 필요할 수 있습니다.")
    
    # 흡연 관련 권고사항
    if is_smoker:
        if primary_prediction in ['Benign', 'Malignant']:
            recommendations['smoking_advice'].append("🚭 즉시 금연하시기 바랍니다.")
            recommendations['smoking_advice'].append("금연 클리닉이나 금연 상담 프로그램을 이용하세요.")
        else:  # Normal but risk exists
            if risk_score > 15:
                recommendations['smoking_advice'].append("🚭 현재는 정상이지만 흡연으로 인한 위험이 있습니다. 금연을 강력히 권합니다.")
            else:
                recommendations['smoking_advice'].append("🚭 건강한 폐를 유지하기 위해 금연을 권장합니다.")
        
        recommendations['smoking_advice'].append("금연 후에도 정기적인 폐 검진이 필요합니다.")
    else:
        if primary_prediction == 'Normal' and risk_score < 10:
            recommendations['smoking_advice'].append("👍 비흡연자로서 좋은 습관을 유지하고 계시는군요!")
        recommendations['smoking_advice'].append("앞으로도 흡연하지 마시기 바랍니다.")
    
    # 운동 관련 권고사항
    if exercise_frequency < 2:  # 운동을 안 하거나 가끔만 하는 경우
        if primary_prediction in ['Benign', 'Malignant']:
            recommendations['exercise_advice'].append("🏃‍♂️ 규칙적인 유산소 운동이 회복에 도움이 됩니다.")
            recommendations['exercise_advice'].append("주 3-4회, 30분씩 걷기나 가벼운 조깅을 시작하세요.")
        else:
            recommendations['exercise_advice'].append("🏃‍♂️ 폐 건강을 위해 규칙적인 운동을 시작하시기 바랍니다.")
            recommendations['exercise_advice'].append("주 3회 이상, 30분씩 유산소 운동을 권장합니다.")
        
        recommendations['exercise_advice'].append("수영, 걷기, 자전거 타기 등이 폐 기능 향상에 좋습니다.")
    else:
        recommendations['exercise_advice'].append("👍 규칙적인 운동 습관이 훌륭합니다!")
        if primary_prediction in ['Benign', 'Malignant']:
            recommendations['exercise_advice'].append("현재 운동량을 유지하되, 무리하지 마시기 바랍니다.")
    
    # 생활습관 권고사항
    recommendations['lifestyle_advice'].append("🍎 항산화 성분이 풍부한 과일과 채소를 충분히 섭취하세요.")
    recommendations['lifestyle_advice'].append("💧 충분한 수분 섭취를 하시기 바랍니다.")
    recommendations['lifestyle_advice'].append("😴 충분한 수면을 취하시기 바랍니다.")
    
    if primary_prediction in ['Benign', 'Malignant']:
        recommendations['lifestyle_advice'].append("🏭 미세먼지가 심한 날에는 외출을 자제하고 마스크를 착용하세요.")
        recommendations['lifestyle_advice'].append("🧘‍♀️ 스트레스 관리를 위한 명상이나 요가를 고려해보세요.")
    
    # 나이에 따른 추가 권고사항
    if age >= 50:
        recommendations['followup_advice'].append("50세 이상이므로 정기적인 폐암 검진이 중요합니다.")
        if is_smoker or primary_prediction != 'Normal':
            recommendations['followup_advice'].append("고위험군에 해당하므로 저선량 흉부 CT 검사를 고려하세요.")
    
    return recommendations

def lung_preprocessing_flask(image_path):
    """
    Flask용 개선된 CT 이미지 폐 전처리 함수
    """
    
    # 이미지 로드
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None, None
    
    original_normalized = img.astype(np.float32) / 255.0
    img_height, img_width = original_normalized.shape
    center_x, center_y = img_width // 2, img_height // 2
    
    # 1. 몸통 추출
    img_hu_approx = (original_normalized * 2000) - 1000
    body_mask = img_hu_approx > -100
    
    kernel = np.ones((5, 5), np.uint8)
    body_mask = cv2.morphologyEx(body_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    
    # 몸통 선택 (중앙에 가장 가까운 큰 영역)
    contours, _ = cv2.findContours(body_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        min_body_area = body_mask.size * 0.15
        max_body_area = body_mask.size * 0.8
        
        best_contour = None
        min_distance = float('inf')
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if min_body_area <= area <= max_body_area:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    distance = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
                    
                    if distance < min_distance:
                        min_distance = distance
                        best_contour = contour
        
        if best_contour is not None:
            body_mask = np.zeros_like(body_mask)
            cv2.fillPoly(body_mask, [best_contour], 1)
        else:
            return None, None, None
    else:
        return None, None, None
    
    # 2. 베드 영역 제거
    bed_mask = np.zeros_like(body_mask, dtype=bool)
    
    # 하단 15%, 좌우 8% 제거
    bottom_region = int(img_height * 0.15)
    side_margin = int(img_width * 0.08)
    bed_mask[-bottom_region:, :] = True
    bed_mask[:, :side_margin] = True
    bed_mask[:, -side_margin:] = True
    
    # 몸통 외부 어두운 영역 제거
    expanded_body = cv2.dilate(body_mask, np.ones((15, 15), np.uint8), iterations=2)
    outside_body = ~expanded_body.astype(bool)
    dark_areas = original_normalized < 0.1
    bed_mask = bed_mask | (outside_body & dark_areas)
    
    # 3. 하얀 조직 및 혈관 제거
    enhanced = np.clip(original_normalized * 1.2 + 0.1, 0, 1)
    white_mask = (enhanced * body_mask) > 0.75
    
    # 혈관/염증 영역 제거 (너무 밝은 부분)
    vessel_mask = (original_normalized > 0.65) & body_mask.astype(bool)
    
    # 폐 영역 마스크 생성 (더 엄격한 밝기 범위)
    intensity_mask = (original_normalized >= 0.18) & (original_normalized <= 0.5)
    lung_mask = body_mask.astype(bool) & ~white_mask & ~bed_mask & ~vessel_mask & intensity_mask
    
    # 4. 몸통 경계 수축
    eroded_body = cv2.erode(body_mask, np.ones((7, 7), np.uint8), iterations=2)
    lung_mask = lung_mask & eroded_body.astype(bool)
    
    # 5. 폐 영역 선택 (좌우 대칭성 고려)
    contours, _ = cv2.findContours(lung_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    final_mask = np.zeros_like(lung_mask, dtype=np.uint8)
    body_area = np.sum(body_mask)
    min_area = body_area * 0.005
    max_area = body_area * 0.35
    
    # 유효한 폐 영역 찾기
    valid_regions = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area <= area <= max_area:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                
                # 위치 검증
                if (cy < img_height * 0.7 and 
                    img_width * 0.15 < cx < img_width * 0.85):
                    center_distance = abs(cx - center_x) / (img_width / 2)
                    valid_regions.append((contour, area, cx, cy, center_distance))
    
    # 좌우 폐 선택
    if len(valid_regions) >= 2:
        # 면적 순 정렬
        valid_regions.sort(key=lambda x: x[1], reverse=True)
        
        # 좌우 분리된 영역 찾기
        selected = [valid_regions[0]]  # 가장 큰 영역
        
        for i in range(1, min(len(valid_regions), 4)):
            region1, region2 = selected[0], valid_regions[i]
            cx1, cx2 = region1[2], region2[2]
            
            # 좌우 분리 확인
            if abs(cx1 - cx2) > img_width * 0.1:
                area_ratio = min(region1[1], region2[1]) / max(region1[1], region2[1])
                if area_ratio > 0.15:  # 면적 비율 검증
                    selected.append(region2)
                    break
        
        # 좌우 균형 검증
        left_count = sum(1 for r in selected if r[2] < center_x)
        right_count = sum(1 for r in selected if r[2] > center_x)
        
        if left_count == 0 or right_count == 0:
            # 균형이 안 맞으면 재선택
            for i in range(min(len(valid_regions), 3)):
                for j in range(i+1, min(len(valid_regions), 4)):
                    r1, r2 = valid_regions[i], valid_regions[j]
                    if (r1[2] < center_x < r2[2]) or (r2[2] < center_x < r1[2]):
                        selected = [r1, r2]
                        break
                if len(selected) == 2:
                    break
        
        # 선택된 영역 마스크에 추가
        for region in selected:
            cv2.fillPoly(final_mask, [region[0]], 1)
            
    elif len(valid_regions) == 1:
        cv2.fillPoly(final_mask, [valid_regions[0][0]], 1)
    
    # 최종 결과
    lung_mask_final = final_mask.astype(bool)
    lung_image = original_normalized * lung_mask_final
    
    return lung_mask_final, lung_image, original_normalized


def preprocess_image_with_improved_segmentation(image_path):
    """
    개선된 이미지 전처리 함수 (Flask 인터페이스와 호환)
    """
    
    try:
        # 개선된 폐 전처리 적용
        lung_mask, lung_img, original_img = lung_preprocessing_flask(image_path)
        
        if lung_mask is None:
            return None, None
        
        # 폐 비율 검증
        lung_ratio = np.sum(lung_mask) / lung_mask.size
        if not (0.003 < lung_ratio < 0.45):
            print(f"폐 비율 검증 실패: {lung_ratio:.3f}")
            return None, None
        
        # 512x512로 리사이즈
        processed_img = cv2.resize(lung_img, (512, 512))
        original_resized = cv2.resize(original_img, (512, 512))
        
        # 정규화 (0-1 범위)
        processed_img = processed_img.astype(np.float32)
        original_resized = original_resized.astype(np.float32)
        
        return processed_img, original_resized
        
    except Exception as e:
        print(f"전처리 오류: {e}")
        return None, None

def allowed_file(filename):
    """허용된 파일 확장자 확인"""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def build_focusnetlc(input_shape=(IMG_SIZE, IMG_SIZE, 1), meta_dim=2):
    """FocusNet-LC 모델 구조 정의 (학습 시와 동일)"""
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
    x = Conv2D(64, (3, 3),strides=2, activation='relu', padding='same')(x)
    x = MaxPooling2D(2)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(2)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(2)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Flatten()(x)

    meta_input = Input(shape=(meta_dim,))
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(3, activation='softmax')(x)
    
    model = Model(inputs=[img_input, meta_input], outputs=output)
    return model

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    """Grad-CAM 히트맵 생성"""
    preds = model.predict(img_array)
    pred_index = tf.argmax(preds[0])
    
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, pred_index]
    
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def create_gradcam_image(original_img, heatmap, alpha=0.6):
    """Grad-CAM 오버레이 이미지 생성 (원본 이미지에 히트맵 오버레이)"""
    # 히트맵을 원본 이미지 크기에 맞게 리사이즈
    heatmap_resized = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    
    # 히트맵을 컬러맵으로 변환 (jet 컬러맵 사용)
    heatmap_colored = plt.cm.jet(heatmap_resized)[..., :3]
    
    # 원본 이미지를 3채널로 변환 (그레이스케일을 RGB로)
    if len(original_img.shape) == 2:
        original_img_3ch = np.stack([original_img]*3, axis=-1)
    else:
        original_img_3ch = original_img
    
    # 원본 이미지 정규화 (0-1 범위로)
    original_img_normalized = original_img_3ch / np.max(original_img_3ch) if np.max(original_img_3ch) > 0 else original_img_3ch
    
    # 히트맵에서 투명도 마스크 생성 (히트맵 값이 낮은 곳은 투명하게)
    transparency_mask = heatmap_resized
    transparency_mask = np.stack([transparency_mask]*3, axis=-1)
    
    # 알파 블렌딩: 히트맵 강도에 따라 투명도 조절
    # 히트맵이 강한 곳은 더 불투명하게, 약한 곳은 더 투명하게
    dynamic_alpha = alpha * transparency_mask
    
    # 최종 이미지 합성
    superimposed_img = (heatmap_colored * dynamic_alpha + 
                       original_img_normalized * (1 - dynamic_alpha))
    
    # 0-255 범위로 변환
    superimposed_img = np.uint8(255 * superimposed_img)
    
    return superimposed_img

# 모델 로드
print("🔄 모델 로딩 중...")
try:
    # 먼저 모델 구조 생성
    model = build_focusnetlc()
    # 가중치 로드
    model.load_weights(h5_path)
    
    print("✅ 모델 로드 완료!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    model = None

@app.route('/')
def index():
    """메인 페이지"""
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    """이미지 예측 및 Grad-CAM 생성"""
    if 'file' not in request.files:
        return jsonify({'error': '파일이 선택되지 않았습니다.'})
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': '파일이 선택되지 않았습니다.'})
    
    if file and allowed_file(file.filename):
        try:
            # 파일 저장
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            
            # 개선된 이미지 전처리 (원본 이미지도 함께 반환)
            processed_img, original_img = preprocess_image_with_improved_segmentation(filepath)
            if processed_img is None:
                # 전처리 실패 시 관리자에게 알림 메시지
                return jsonify({
                    'error': '전처리 실패',
                    'message': '이 CT 이미지는 자동 분석이 어려워 관리자에게 전송되었습니다. 수동 검토 후 결과를 알려드리겠습니다.',
                    'admin_notification': True
                })
            
            # 모델 입력 형태로 변환
            img_array = processed_img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
            meta_array = np.array([[0.1, 0.1]])  # 더미 메타데이터
            
            # 예측
            predictions = model.predict([img_array, meta_array])
            pred_class = np.argmax(predictions[0])
            confidence = float(predictions[0][pred_class])
            
            # Grad-CAM 생성
            count =0
            last_conv_layer = None
            for layer in reversed(model.layers):
                if isinstance(layer, tf.keras.layers.Conv2D):
                    count = count +1
                    if(count >= 5) : 
                        last_conv_layer = layer.name
                        break
            
            if last_conv_layer:
                heatmap = make_gradcam_heatmap([img_array, meta_array], model, last_conv_layer)
                # 원본 이미지에 Grad-CAM 오버레이
                gradcam_img = create_gradcam_image(original_img, heatmap)
                
                # 이미지를 base64로 인코딩
                _, buffer = cv2.imencode('.png', gradcam_img)
                gradcam_b64 = base64.b64encode(buffer).decode('utf-8')
            else:
                gradcam_b64 = None
            
            # 원본 이미지도 base64로 인코딩
            _, orig_buffer = cv2.imencode('.png', original_img * 255)
            original_b64 = base64.b64encode(orig_buffer).decode('utf-8')
            
            # 전처리된 이미지도 base64로 인코딩
            _, proc_buffer = cv2.imencode('.png', processed_img * 255)
            processed_b64 = base64.b64encode(proc_buffer).decode('utf-8')
            
            # 결과 반환
            result = {
                'prediction': CLASS_NAMES[pred_class],
                'confidence': round(confidence * 100, 2),
                'color': CLASS_COLORS[pred_class],
                'all_predictions': {
                    CLASS_NAMES[i]: round(float(predictions[0][i]) * 100, 2) 
                    for i in range(len(CLASS_NAMES))
                },
                'original_image': f"data:image/png;base64,{original_b64}",
                'processed_image': f"data:image/png;base64,{processed_b64}",
                'gradcam_image': f"data:image/png;base64,{gradcam_b64}" if gradcam_b64 else None,
                'preprocessing_success': True,
                'need_survey': True  # 설문조사 필요 플래그
            }
            
            # 임시 파일 삭제
            os.remove(filepath)
            
            return jsonify(result)
            
        except Exception as e:
            # 임시 파일 삭제 (실패 시에도)
            if 'filepath' in locals() and os.path.exists(filepath):
                os.remove(filepath)
            
            return jsonify({
                'error': '처리 중 오류 발생',
                'message': f'시스템 오류가 발생했습니다: {str(e)}',
                'admin_notification': True
            })
    
    return jsonify({'error': '잘못된 파일 형식입니다.'})

@app.route('/health_recommendations', methods=['POST'])
def health_recommendations():
    """설문조사 결과를 받아 개인화된 건강 권고사항 제공"""
    try:
        data = request.get_json()
        
        # 진단 결과
        prediction_result = data.get('prediction_result', {})
        
        # 설문조사 결과
        survey_data = {
            'smoking': data.get('smoking', False),
            'exercise': int(data.get('exercise', 0)),
            'age': int(data.get('age', 40))
        }
        
        # 건강 권고사항 생성
        recommendations = generate_health_recommendations(prediction_result, survey_data)
        
        return jsonify({
            'success': True,
            'recommendations': recommendations
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'권고사항 생성 중 오류 발생: {str(e)}'
        })

# HTML 템플릿 (설문조사 및 건강 권고사항 기능 추가)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>FocusNet-LC 폐암 진단</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        
        .container {
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0,0,0,0.1);
            overflow: hidden;
        }
        
        .header {
            background: linear-gradient(135deg, #2c3e50, #3498db);
            color: white;
            padding: 30px;
            text-align: center;
        }
        
        .header h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
        }
        
        .header p {
            font-size: 1.2em;
            opacity: 0.9;
        }
        
        .content {
            padding: 40px;
        }
        
        .upload-section {
            text-align: center;
            margin-bottom: 40px;
        }
        
        .file-input-wrapper {
            position: relative;
            display: inline-block;
            margin: 20px 0;
        }
        
        .file-input {
            display: none;
        }
        
        .file-input-button {
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 15px 30px;
            border: none;
            border-radius: 50px;
            font-size: 1.1em;
            cursor: pointer;
            transition: all 0.3s;
        }
        
        .file-input-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(0,0,0,0.2);
        }
        
        .predict-button {
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 15px 40px;
            border: none;
            border-radius: 50px;
            font-size: 1.2em;
            cursor: pointer;
            margin-left: 20px;
            transition: all 0.3s;
        }
        
        .predict-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(0,0,0,0.2);
        }
        
        .predict-button:disabled {
            background: #ccc;
            cursor: not-allowed;
            transform: none;
            box-shadow: none;
        }
        
        .results-section {
            display: none;
            margin-top: 40px;
        }
        
        .prediction-card {
            background: #f8f9fa;
            border-radius: 15px;
            padding: 30px;
            margin-bottom: 30px;
            text-align: center;
        }
        
        .prediction-result {
            font-size: 2em;
            font-weight: bold;
            margin-bottom: 10px;
        }
        
        .confidence {
            font-size: 1.5em;
            margin-bottom: 20px;
        }
        
        .all-predictions {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 20px;
        }
        
        .prediction-item {
            background: white;
            padding: 15px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        }
        
        .images-container {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
            gap: 30px;
            margin-top: 30px;
        }
        
        .image-card {
            background: #f8f9fa;
            border-radius: 15px;
            padding: 20px;
            text-align: center;
        }
        
        .image-card h3 {
            margin-bottom: 15px;
            color: #333;
        }
        
        .image-card img {
            max-width: 100%;
            height: auto;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0,0,0,0.1);
        }
        
        .survey-section {
            display: none;
            background: #f8f9fa;
            border-radius: 15px;
            padding: 30px;
            margin-top: 30px;
        }
        
        .survey-title {
            font-size: 1.5em;
            margin-bottom: 20px;
            text-align: center;
            color: #333;
        }
        
        .survey-form {
            max-width: 600px;
            margin: 0 auto;
        }
        
        .form-group {
            margin-bottom: 25px;
        }
        
        .form-group label {
            display: block;
            font-weight: bold;
            margin-bottom: 10px;
            color: #333;
        }
        
        .form-group input[type="radio"] {
            margin-right: 10px;
        }
        
        .form-group .radio-group {
            display: flex;
            flex-direction: column;
            gap: 10px;
        }
        
        .form-group .radio-option {
            display: flex;
            align-items: center;
            padding: 10px;
            background: white;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s;
        }
        
        .form-group .radio-option:hover {
            background: #e9ecef;
        }
        
        .form-group input[type="number"] {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-size: 1em;
        }
        
        .submit-survey-button {
            background: linear-gradient(135deg, #17a2b8, #138496);
            color: white;
            padding: 15px 40px;
            border: none;
            border-radius: 50px;
            font-size: 1.1em;
            cursor: pointer;
            width: 100%;
            transition: all 0.3s;
        }
        
        .submit-survey-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(0,0,0,0.2);
        }
        
        .recommendations-section {
            display: none;
            margin-top: 30px;
        }
        
        .recommendations-card {
            background: #f8f9fa;
            border-radius: 15px;
            padding: 30px;
            margin-bottom: 20px;
        }
        
        .priority-banner {
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
            text-align: center;
            font-weight: bold;
            font-size: 1.2em;
        }
        
        .priority-low {
            background: #d4edda;
            color: #155724;
            border: 2px solid #c3e6cb;
        }
        
        .priority-medium {
            background: #fff3cd;
            color: #856404;
            border: 2px solid #ffeaa7;
        }
        
        .priority-high {
            background: #f8d7da;
            color: #721c24;
            border: 2px solid #f5c6cb;
        }
        
        .priority-urgent {
            background: #f8d7da;
            color: #721c24;
            border: 2px solid #f5c6cb;
            animation: pulse 2s infinite;
        }
        
        @keyframes pulse {
            0% { transform: scale(1); }
            50% { transform: scale(1.02); }
            100% { transform: scale(1); }
        }
        
        .recommendation-category {
            margin-bottom: 25px;
        }
        
        .recommendation-category h4 {
            color: #333;
            margin-bottom: 15px;
            padding-bottom: 8px;
            border-bottom: 2px solid #ddd;
        }
        
        .recommendation-list {
            list-style: none;
            padding: 0;
        }
        
        .recommendation-list li {
            background: white;
            padding: 12px 15px;
            margin-bottom: 8px;
            border-radius: 8px;
            border-left: 4px solid #007bff;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        
        .recommendation-list li:before {
            content: "✓ ";
            color: #28a745;
            font-weight: bold;
            margin-right: 8px;
        }
        
        .smoking-advice li {
            border-left-color: #dc3545;
        }
        
        .exercise-advice li {
            border-left-color: #28a745;
        }
        
        .medical-advice li {
            border-left-color: #ffc107;
        }
        
        .lifestyle-advice li {
            border-left-color: #17a2b8;
        }
        
        .followup-advice li {
            border-left-color: #6f42c1;
        }
        
        .loading {
            display: none;
            text-align: center;
            margin: 20px 0;
        }
        
        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #3498db;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
            margin: 0 auto 20px;
        }
        
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        
        .error-message {
            background: #dc3545;
            color: white;
            padding: 15px;
            border-radius: 10px;
            margin: 20px 0;
            display: none;
        }
        
        .admin-message {
            background: #ffc107;
            color: #856404;
            padding: 20px;
            border-radius: 10px;
            margin: 20px 0;
            display: none;
            text-align: center;
        }
        
        .admin-message h4 {
            margin-bottom: 10px;
            color: #856404;
        }
        
        @media (max-width: 768px) {
            .images-container {
                grid-template-columns: 1fr;
            }
            
            .predict-button {
                margin-left: 0;
                margin-top: 10px;
            }
            
            .form-group .radio-group {
                gap: 8px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>AI Bread Scan</h1>
            <p>AI 기반 폐암 진단 시스템 (서강대 AISW 텐서플로 활용기초)</p>
        </div>
        
        <div class="content">
            <div class="upload-section">
                <h2>CT 이미지를 업로드하세요</h2>
                <div class="file-input-wrapper">
                    <input type="file" id="fileInput" class="file-input" accept=".png,.jpg,.jpeg">
                    <button class="file-input-button" onclick="document.getElementById('fileInput').click()">
                        📁 파일 선택
                    </button>
                </div>
                <button class="predict-button" id="predictButton" disabled onclick="predict()">
                    🔍 진단 시작
                </button>
                <div id="fileName" style="margin-top: 10px; color: #666;"></div>
            </div>
            
            <div class="loading" id="loading">
                <div class="spinner"></div>
                <p>AI가 이미지를 분석하고 있습니다...</p>
            </div>
            
            <div class="error-message" id="errorMessage"></div>
            
            <div class="admin-message" id="adminMessage">
                <h4>⚠️ 전처리 실패</h4>
                <p id="adminMessageText"></p>
            </div>
            
            <div class="results-section" id="resultsSection">
                <div class="prediction-card">
                    <div class="prediction-result" id="predictionResult"></div>
                    <div class="confidence" id="confidence"></div>
                    <div class="all-predictions" id="allPredictions"></div>
                </div>
                
                <div class="images-container">
                    <div class="image-card">
                        <h3>📷 원본 이미지</h3>
                        <img id="originalImage" src="" alt="원본 이미지">
                    </div>
                    <div class="image-card">
                        <h3>🔍 전처리된 이미지</h3>
                        <img id="processedImage" src="" alt="전처리된 이미지">
                    </div>
                    <div class="image-card">
                        <h3>🎯 Grad-CAM 히트맵</h3>
                        <img id="gradcamImage" src="" alt="Grad-CAM 히트맵">
                    </div>
                </div>
            </div>
            
            <!-- 설문조사 섹션 -->
            <div class="survey-section" id="surveySection">
                <h3 class="survey-title">📋 건강 설문조사</h3>
                <p style="text-align: center; margin-bottom: 30px; color: #666;">
                    더 정확한 건강 권고사항을 위해 간단한 설문에 참여해주세요.
                </p>
                
                <form class="survey-form" id="surveyForm">
                    <div class="form-group">
                        <label>1. 현재 흡연 상태는 어떠신가요?</label>
                        <div class="radio-group">
                            <label class="radio-option">
                                <input type="radio" name="smoking" value="false" required>
                                <span>🚭 비흡연자 (흡연한 적 없음)</span>
                            </label>
                            <label class="radio-option">
                                <input type="radio" name="smoking" value="false" required>
                                <span>🚫 금연자 (과거 흡연, 현재 금연)</span>
                            </label>
                            <label class="radio-option">
                                <input type="radio" name="smoking" value="true" required>
                                <span>🚬 현재 흡연자</span>
                            </label>
                        </div>
                    </div>
                    
                    <div class="form-group">
                        <label>2. 평소 운동은 얼마나 자주 하시나요?</label>
                        <div class="radio-group">
                            <label class="radio-option">
                                <input type="radio" name="exercise" value="0" required>
                                <span>😴 운동을 거의 하지 않음</span>
                            </label>
                            <label class="radio-option">
                                <input type="radio" name="exercise" value="1" required>
                                <span>🚶‍♂️ 가끔 운동함 (주 1-2회)</span>
                            </label>
                            <label class="radio-option">
                                <input type="radio" name="exercise" value="2" required>
                                <span>🏃‍♂️ 정기적으로 운동함 (주 3회 이상)</span>
                            </label>
                        </div>
                    </div>
                    
                    <div class="form-group">
                        <label for="age">3. 연령을 입력해주세요:</label>
                        <input type="number" id="age" name="age" min="1" max="120" value="40" required>
                    </div>
                    
                    <button type="submit" class="submit-survey-button">
                        📊 건강 권고사항 받기
                    </button>
                </form>
            </div>
            
            <!-- 건강 권고사항 섹션 -->
            <div class="recommendations-section" id="recommendationsSection">
                <div class="recommendations-card">
                    <h3 style="text-align: center; margin-bottom: 20px; color: #333;">
                        💡 개인화된 건강 권고사항
                    </h3>
                    
                    <div id="priorityBanner" class="priority-banner"></div>
                    
                    <div id="medicalAdvice" class="recommendation-category">
                        <h4>🏥 의료 관련 권고사항</h4>
                        <ul class="recommendation-list medical-advice"></ul>
                    </div>
                    
                    <div id="smokingAdvice" class="recommendation-category">
                        <h4>🚭 흡연 관련 권고사항</h4>
                        <ul class="recommendation-list smoking-advice"></ul>
                    </div>
                    
                    <div id="exerciseAdvice" class="recommendation-category">
                        <h4>🏃‍♂️ 운동 관련 권고사항</h4>
                        <ul class="recommendation-list exercise-advice"></ul>
                    </div>
                    
                    <div id="lifestyleAdvice" class="recommendation-category">
                        <h4>🌱 생활습관 권고사항</h4>
                        <ul class="recommendation-list lifestyle-advice"></ul>
                    </div>
                    
                    <div id="followupAdvice" class="recommendation-category">
                        <h4>📅 추후 관리 계획</h4>
                        <ul class="recommendation-list followup-advice"></ul>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <script>
        let currentPredictionResult = null;
        
        document.getElementById('fileInput').addEventListener('change', function(e) {
            const file = e.target.files[0];
            const predictButton = document.getElementById('predictButton');
            const fileName = document.getElementById('fileName');
            
            if (file) {
                fileName.textContent = `선택된 파일: ${file.name}`;
                predictButton.disabled = false;
            } else {
                fileName.textContent = '';
                predictButton.disabled = true;
            }
        });
        
        document.getElementById('surveyForm').addEventListener('submit', function(e) {
            e.preventDefault();
            submitSurvey();
        });
        
        function showError(message) {
            const errorDiv = document.getElementById('errorMessage');
            errorDiv.textContent = message;
            errorDiv.style.display = 'block';
            setTimeout(() => {
                errorDiv.style.display = 'none';
            }, 10000);
        }
        
        function showAdminMessage(message) {
            const adminDiv = document.getElementById('adminMessage');
            const adminText = document.getElementById('adminMessageText');
            adminText.textContent = message;
            adminDiv.style.display = 'block';
            
            setTimeout(() => {
                adminDiv.style.display = 'none';
            }, 15000);
        }
        
        async function predict() {
            const fileInput = document.getElementById('fileInput');
            const file = fileInput.files[0];
            
            if (!file) {
                showError('파일을 선택해주세요.');
                return;
            }
            
            // UI 상태 변경
            document.getElementById('loading').style.display = 'block';
            document.getElementById('resultsSection').style.display = 'none';
            document.getElementById('surveySection').style.display = 'none';
            document.getElementById('recommendationsSection').style.display = 'none';
            document.getElementById('errorMessage').style.display = 'none';
            document.getElementById('adminMessage').style.display = 'none';
            document.getElementById('predictButton').disabled = true;
            
            const formData = new FormData();
            formData.append('file', file);
            
            try {
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
                
                const result = await response.json();
                
                if (result.error) {
                    if (result.admin_notification) {
                        showAdminMessage(result.message || result.error);
                    } else {
                        showError(result.error);
                    }
                    return;
                }
                
                // 현재 예측 결과 저장
                currentPredictionResult = result;
                
                // 결과 표시
                document.getElementById('predictionResult').textContent = result.prediction;
                document.getElementById('predictionResult').style.color = result.color;
                document.getElementById('confidence').textContent = `신뢰도: ${result.confidence}%`;
                
                // 모든 예측 결과 표시
                const allPredDiv = document.getElementById('allPredictions');
                allPredDiv.innerHTML = '';
                for (const [className, confidence] of Object.entries(result.all_predictions)) {
                    const div = document.createElement('div');
                    div.className = 'prediction-item';
                    div.innerHTML = `<strong>${className}</strong><br>${confidence}%`;
                    allPredDiv.appendChild(div);
                }
                
                // 이미지 표시
                document.getElementById('originalImage').src = result.original_image;
                document.getElementById('processedImage').src = result.processed_image;
                if (result.gradcam_image) {
                    document.getElementById('gradcamImage').src = result.gradcam_image;
                }
                
                document.getElementById('resultsSection').style.display = 'block';
                
                // 설문조사 표시
                if (result.need_survey) {
                    setTimeout(() => {
                        document.getElementById('surveySection').style.display = 'block';
                        document.getElementById('surveySection').scrollIntoView({ 
                            behavior: 'smooth', 
                            block: 'start' 
                        });
                    }, 1000);
                }
                
            } catch (error) {
                showError('서버 연결 오류가 발생했습니다.');
                console.error('Error:', error);
            } finally {
                document.getElementById('loading').style.display = 'none';
                document.getElementById('predictButton').disabled = false;
            }
        }
        
        async function submitSurvey() {
            if (!currentPredictionResult) {
                showError('진단 결과가 없습니다. 먼저 이미지 진단을 수행해주세요.');
                return;
            }
            
            const formData = new FormData(document.getElementById('surveyForm'));
            const surveyData = {
                prediction_result: currentPredictionResult,
                smoking: formData.get('smoking') === 'true',
                exercise: parseInt(formData.get('exercise')),
                age: parseInt(formData.get('age'))
            };
            
            try {
                const response = await fetch('/health_recommendations', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify(surveyData)
                });
                
                const result = await response.json();
                
                if (result.success) {
                    displayRecommendations(result.recommendations);
                } else {
                    showError(result.error || '권고사항 생성 중 오류가 발생했습니다.');
                }
                
            } catch (error) {
                showError('서버 연결 오류가 발생했습니다.');
                console.error('Error:', error);
            }
        }
        
        function displayRecommendations(recommendations) {
            // 우선순위 배너 설정
            const priorityBanner = document.getElementById('priorityBanner');
            const priorityTexts = {
                'low': '✅ 저위험군 - 예방 관리 중심',
                'medium': '⚠️ 중위험군 - 적극적 관리 필요',
                'high': '🚨 고위험군 - 신속한 대응 필요',
                'urgent': '🆘 긴급 - 즉시 의료진 상담 필요'
            };
            
            priorityBanner.textContent = priorityTexts[recommendations.priority];
            priorityBanner.className = `priority-banner priority-${recommendations.priority}`;
            
            // 각 카테고리별 권고사항 표시
            const categories = [
                { id: 'medicalAdvice', key: 'medical_advice' },
                { id: 'smokingAdvice', key: 'smoking_advice' },
                { id: 'exerciseAdvice', key: 'exercise_advice' },
                { id: 'lifestyleAdvice', key: 'lifestyle_advice' },
                { id: 'followupAdvice', key: 'followup_advice' }
            ];
            
            categories.forEach(category => {
                const adviceList = recommendations[category.key] || [];
                const listElement = document.querySelector(`#${category.id} .recommendation-list`);
                
                if (adviceList.length > 0) {
                    listElement.innerHTML = '';
                    adviceList.forEach(advice => {
                        const li = document.createElement('li');
                        li.textContent = advice;
                        listElement.appendChild(li);
                    });
                    document.getElementById(category.id).style.display = 'block';
                } else {
                    document.getElementById(category.id).style.display = 'none';
                }
            });
            
            // 권고사항 섹션 표시 및 스크롤
            document.getElementById('recommendationsSection').style.display = 'block';
            setTimeout(() => {
                document.getElementById('recommendationsSection').scrollIntoView({ 
                    behavior: 'smooth', 
                    block: 'start' 
                });
            }, 300);
        }
    </script>
</body>
</html>
"""

# 템플릿 폴더 생성 및 HTML 파일 저장
template_dir = 'templates'
os.makedirs(template_dir, exist_ok=True)
with open(os.path.join(template_dir, 'index.html'), 'w', encoding='utf-8') as f:
    f.write(HTML_TEMPLATE)

if __name__ == '__main__':
    if model is None:
        print("❌ 모델을 로드할 수 없어 서버를 시작할 수 없습니다.")
    else:
        print("🚀 서버 시작 중...")
        print("📝 개선된 전처리 알고리즘이 적용되었습니다.")
        print("🎯 Grad-CAM이 원본 이미지에 오버레이됩니다.")
        print("⚠️  전처리 실패 시 관리자 알림 기능이 활성화되었습니다.")
        print("📋 설문조사 및 개인화된 건강 권고사항 기능이 추가되었습니다.")
        print("💡 주요 개선사항:")
        print("   - Normal 진단 시에도 다른 확률 고려한 위험도 평가")
        print("   - 흡연/운동 습관에 따른 맞춤형 건강 권고")
        print("   - Benign/Malignant 진단 시 즉시 의료진 상담 권유")
        print("   - 연령대별 차별화된 검진 주기 안내")
        print("   - 우선순위별 시각적 구분 (저/중/고/긴급)")
        # 실제 배포 시에는 host를 '0.0.0.0'으로 설정
        app.run(host='localhost', port=8001, debug=False)

🔄 모델 로딩 중...
✅ 모델 로드 완료!
🚀 서버 시작 중...
📝 개선된 전처리 알고리즘이 적용되었습니다.
🎯 Grad-CAM이 원본 이미지에 오버레이됩니다.
⚠️  전처리 실패 시 관리자 알림 기능이 활성화되었습니다.
📋 설문조사 및 개인화된 건강 권고사항 기능이 추가되었습니다.
💡 주요 개선사항:
   - Normal 진단 시에도 다른 확률 고려한 위험도 평가
   - 흡연/운동 습관에 따른 맞춤형 건강 권고
   - Benign/Malignant 진단 시 즉시 의료진 상담 권유
   - 연령대별 차별화된 검진 주기 안내
   - 우선순위별 시각적 구분 (저/중/고/긴급)
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8001
Press CTRL+C to quit
127.0.0.1 - - [10/Jun/2025 21:39:21] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 29ms/step


127.0.0.1 - - [10/Jun/2025 21:39:33] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2025 21:40:05] "POST /health_recommendations HTTP/1.1" 200 -


1/1 [==============================] - 0s 25ms/step


127.0.0.1 - - [10/Jun/2025 21:46:22] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2025 21:46:27] "POST /health_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2025 21:46:54] "POST /health_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2025 21:47:01] "POST /health_recommendations HTTP/1.1" 200 -
